<a href="https://colab.research.google.com/github/sula316/ML_MY_Project_FCUT/blob/main/ML_MY_Project_FCUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stept_1_Data_Preprocessing

##工具

In [ ]:
!pip install numpy pandas matplotlib seaborn scikit-learn scipy > log.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy

##1.讀檔>>去除取樣資料的檔頭>>取22~30023

根據檔案內容的初步檢查，數據在第二個 `***End_of_Header***` 標記之後開始。我們將解析檔案，找到實際數據開始的行，然後使用 `pandas` 讀取並篩選指定的行範圍（第22到第30023行）。

In [30]:
import pandas as pd

file_path = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_01.lvm'

def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 0-indexed line number of the actual data header
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # After the second '***End_of_Header***'
            # Look for the line containing the data column names
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            # Handle cases where the header might be on the next line after End_of_Header
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError("Could not find the data header line in the file.")
    return data_header_line_index

data_header_0_indexed = get_data_header_line_index(file_path)

print(f"數據列標題行 (0-indexed) 位於: {data_header_0_indexed}")

# 讀取從數據列標題行 (X_Value行) 的下一行開始，直到檔案的最後一行
# 這表示我們將跳過所有標頭行，以及包含列標題的行本身。
skiprows_for_pd_read_csv = data_header_0_indexed + 1

# 使用 pandas 讀取檔案，直接從計算出的行數開始讀取到檔案結束
# 因為我們手動計算了 skiprows，所以 header=None，然後手動設置列名
df_temp = pd.read_csv(
    file_path,
    skiprows=skiprows_for_pd_read_csv,
    sep='\t',
    header=None # 不將任何行解釋為標題行，因為我們已將其跳過
)

# 從文件中讀取實際的列標題，並手動指定給 DataFrame
with open(file_path, 'r', encoding='utf-8') as f:
    header_line_content = f.readlines()[data_header_0_indexed]
# 分割並清理列名中的空格
columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

# 移除 'Comment' 列，因為它看起來是空的或不相關的
if 'Comment' in columns:
    columns.remove('Comment')

# 將處理過的列名分配給 DataFrame
df_temp.columns = columns

processed_df = df_temp.copy()

print("處理後的資料前5行：")
display(processed_df.head())
display(processed_df.tail())
print("處理後的資料形狀：")
print(processed_df.shape)

數據列標題行 (0-indexed) 位於: 22
處理後的資料前5行：


,X_Value,X,Y,Z,Voltage
0,0.000000,0.053857,-0.002894,0.013345,0.000443
1,0.000156,0.032555,0.021578,0.003450,0.002672
2,0.000312,0.004798,-0.005554,0.051723,0.002672
3,0.000469,-0.019530,-0.003355,0.032483,0.002991
4,0.000625,-0.004428,0.011777,0.012203,0.002991


,X_Value,X,Y,Z,Voltage
29995,4.686719,0.009914,0.010303,-0.003439,0.001717
29996,4.686875,0.006148,-0.000258,-0.008753,0.002035
29997,4.687031,0.003009,0.003586,0.000388,0.001398
29998,4.687187,-0.010613,-0.001900,0.007021,-0.000513
29999,4.687344,-0.000942,-0.001543,-0.002888,0.001080


處理後的資料形狀：
(30000, 5)


##2.合併資料>>Healthy的各維度>>各別組成一張Table

In [11]:
import os
import pandas as pd

# Reuse the previously defined function to find the data header line index
def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 0-indexed line number of the actual data header
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # After the second '***End_of_Header***'
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError(f"Could not find the data header line in the file: {file_path}")
    return data_header_line_index

# Define the directory containing the LVM files (same as the previous file path's directory)
data_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/'
lvm_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.lvm')]

all_processed_dfs = []

print(f"Found {len(lvm_files)} LVM files in the directory: {data_dir}\n")

for file_path in lvm_files:
    print(f"Processing file: {file_path}")
    try:
        # Get the 0-indexed line number where data headers (X_Value, X, Y, Z, Voltage) are located
        data_header_0_indexed = get_data_header_line_index(file_path)

        # Calculate skiprows for pandas: skip all header lines + the line containing data column names
        skiprows_for_pd_read_csv = data_header_0_indexed + 1

        # Read from the first data row after the header until the end of the file
        df_temp = pd.read_csv(
            file_path,
            skiprows=skiprows_for_pd_read_csv,
            sep='\t',
            header=None # We will assign column names manually
        )

        # Extract actual column names from the data header line
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line_content = f.readlines()[data_header_0_indexed]
        columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

        # Remove 'Comment' column if present, as it was deemed irrelevant before
        if 'Comment' in columns:
            columns.remove('Comment')

        # Assign column names to the DataFrame
        if len(df_temp.columns) == len(columns):
            df_temp.columns = columns
        else:
            print(f"Warning: Column count mismatch in {file_path}. Expected {len(columns)} columns, but DataFrame has {len(df_temp.columns)}. Skipping this file.\n")
            continue # Skip to the next file if column count doesn't match

        # Extract filename without extension to use as a prefix for columns (except 'X_Value')
        filename_without_ext = os.path.basename(file_path).split('.')[0]
        renamed_cols = {col: f"{filename_without_ext}_{col}" for col in columns if col != 'X_Value'}
        df_temp = df_temp.rename(columns=renamed_cols)

        all_processed_dfs.append(df_temp)

    except Exception as e:
        print(f"Error processing {file_path}: {e}\n")

merged_df = pd.DataFrame()
if all_processed_dfs:
    # Initialize merged_df with the first processed DataFrame.
    # X_Value will be the common column for merging.
    merged_df = all_processed_dfs[0]

    # Merge subsequent DataFrames on 'X_Value' using an outer join to keep all time points
    for i in range(1, len(all_processed_dfs)):
        merged_df = pd.merge(merged_df, all_processed_dfs[i], on='X_Value', how='outer')

    # Sort by 'X_Value' to maintain time series order
    merged_df = merged_df.sort_values(by='X_Value').reset_index(drop=True)

    print("\nAll files processed and merged successfully!")
    print("Merged Data (first 5 rows):")
    display(merged_df.head())
    print("\nMerged Data Shape:")
    print(merged_df.shape)
else:
    print("No dataframes were successfully processed or found to merge.")

Found 48 LVM files in the directory: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/

Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_08.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_02.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_31.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_34.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_32.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_13.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_09.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_04.lvm
Processing file: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/20240930

,X_Value,202409302014_08_X,202409302014_08_Y,202409302014_08_Z,202409302014_08_Voltage,202409302014_02_X,202409302014_02_Y,202409302014_02_Z,202409302014_02_Voltage,202409302014_31_X,...,202409302014_17_Z,202409302014_17_Voltage,202409302014_11_X,202409302014_11_Y,202409302014_11_Z,202409302014_11_Voltage,202409302014_23_X,202409302014_23_Y,202409302014_23_Z,202409302014_23_Voltage
0,0.000000,-0.018021,-0.000935,-0.020087,0.000443,0.006437,-0.004312,0.003345,0.000124,-0.001781,...,-0.005533,-0.040971,-0.016005,-0.001897,-0.020615,0.002035,-0.009129,-0.000674,-0.008132,0.111939
1,0.000156,-0.006662,0.001867,-0.003899,0.000124,0.007665,-0.000951,0.006284,0.001398,0.020879,...,-0.033841,-0.020583,-0.010381,-0.002141,0.006880,0.001398,-0.016798,0.000819,-0.001892,0.092825
2,0.000312,0.013649,0.004049,0.018903,0.000124,-0.000910,0.006552,0.000462,0.001080,0.023079,...,-0.029128,0.002991,-0.005601,0.021896,0.034135,0.001717,0.017786,-0.000157,0.023986,0.070208
3,0.000469,0.020219,0.002618,-0.051073,-0.000195,-0.007906,-0.002169,-0.000201,0.001080,0.023308,...,0.047065,0.003310,0.044207,-0.029773,-0.016545,0.001398,0.011756,0.007153,-0.033423,0.048227
4,0.000625,0.023051,0.012546,-0.062373,-0.000832,0.003494,0.001144,-0.019746,0.000761,0.004017,...,-0.001145,0.002991,0.020391,0.029132,0.044562,0.002354,0.009312,-0.005543,-0.027979,0.025609



Merged Data Shape:
(30000, 193)




> Table大小皆為30000x48、共4份




##3.補缺漏值>>用means>>各別儲存乾淨資料<br>

In [17]:
# 檢查 merged_df 中是否有 NaN 值
print("處理前 NaN 值的數量：")
print(merged_df.isnull().sum().sum())

# 使用每個欄位的平均值填充 NaN 值，'X_Value' 列通常不需要填充因為它是時間軸，但為確保一致性，可以檢查
# 只對非 'X_Value' 的數據列進行填充
data_cols = [col for col in merged_df.columns if col != 'X_Value']
for col in data_cols:
    if merged_df[col].isnull().any():
        merged_df[col] = merged_df[col].fillna(merged_df[col].mean())

# 再次檢查 NaN 值以確保所有缺漏值都已被處理
print("\n處理後 NaN 值的數量：")
print(merged_df.isnull().sum().sum())

print("\n處理後的資料前5行：")
display(merged_df.head())

print("\n處理後的資料形狀：")
print(merged_df.shape)

# 由於原始需求提到「各別儲存乾淨資料」，這裡假設是指儲存這個完整的、已填充缺漏值的 merged_df
# 如果需要將不同測量類型（如X, Y, Z, Voltage）分開儲存，請進一步說明。
# 我們可以將處理後的資料儲存為一個新的CSV檔案，方便後續使用。
output_file_path = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy_merged_cleaned.csv'
merged_df.to_csv(output_file_path, index=False)
print(f"\n乾淨的合併資料已儲存至：{output_file_path}")

處理前 NaN 值的數量：
0

處理後 NaN 值的數量：
0

處理後的資料前5行：


,X_Value,202409302014_08_X,202409302014_08_Y,202409302014_08_Z,202409302014_08_Voltage,202409302014_02_X,202409302014_02_Y,202409302014_02_Z,202409302014_02_Voltage,202409302014_31_X,...,202409302014_17_Z,202409302014_17_Voltage,202409302014_11_X,202409302014_11_Y,202409302014_11_Z,202409302014_11_Voltage,202409302014_23_X,202409302014_23_Y,202409302014_23_Z,202409302014_23_Voltage
0,0.000000,-0.018021,-0.000935,-0.020087,0.000443,0.006437,-0.004312,0.003345,0.000124,-0.001781,...,-0.005533,-0.040971,-0.016005,-0.001897,-0.020615,0.002035,-0.009129,-0.000674,-0.008132,0.111939
1,0.000156,-0.006662,0.001867,-0.003899,0.000124,0.007665,-0.000951,0.006284,0.001398,0.020879,...,-0.033841,-0.020583,-0.010381,-0.002141,0.006880,0.001398,-0.016798,0.000819,-0.001892,0.092825
2,0.000312,0.013649,0.004049,0.018903,0.000124,-0.000910,0.006552,0.000462,0.001080,0.023079,...,-0.029128,0.002991,-0.005601,0.021896,0.034135,0.001717,0.017786,-0.000157,0.023986,0.070208
3,0.000469,0.020219,0.002618,-0.051073,-0.000195,-0.007906,-0.002169,-0.000201,0.001080,0.023308,...,0.047065,0.003310,0.044207,-0.029773,-0.016545,0.001398,0.011756,0.007153,-0.033423,0.048227
4,0.000625,0.023051,0.012546,-0.062373,-0.000832,0.003494,0.001144,-0.019746,0.000761,0.004017,...,-0.001145,0.002991,0.020391,0.029132,0.044562,0.002354,0.009312,-0.005543,-0.027979,0.025609



處理後的資料形狀：
(30000, 193)

乾淨的合併資料已儲存至：/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy_merged_cleaned.csv


###**去除X_Value

In [27]:
import os
import pandas as pd

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'

# 創建輸出目錄如果它不存在
os.makedirs(output_base_dir, exist_ok=True)

print(f"正在將資料按測量類型儲存到: {output_base_dir}")

# 定義要分開儲存的測量類型
measurement_types = ['X', 'Y', 'Z', 'Voltage']

# 確保 merged_df 已經存在且被處理過 (來自之前的步驟)
if 'merged_df' not in locals() or merged_df.empty:
    print("錯誤: merged_df 不存在或為空，請確認之前的步驟已成功執行。")
else:
    for m_type in measurement_types:
        # 找出所有屬於當前測量類型的欄位
        # 並將這些欄位排序，以便輸出時順序一致
        data_cols_for_type = sorted([col for col in merged_df.columns if col.endswith(f'_{m_type}')])

        # 這裡不包含 'X_Value'，因為使用者要求最終儲存的 CSV 檔案中不包含此欄位
        cols_to_save = data_cols_for_type

        # 創建一個新的DataFrame，只包含當前測量類型的所有數據 (不含 X_Value)
        df_m_type = merged_df[cols_to_save].copy()

        # 定義輸出檔案路徑，使用新的命名格式
        output_file_name = f'Healthy_Merge_Clean_Table_{m_type}.csv'
        output_file_path = os.path.join(output_base_dir, output_file_name)

        # 儲存為CSV檔案
        df_m_type.to_csv(output_file_path, index=False)
        print(f"已儲存 {m_type} 類型資料到: {output_file_path} (形狀: {df_m_type.shape})")

print("所有測量類型的資料已分開儲存完畢。")

正在將資料按測量類型儲存到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data
已儲存 X 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_X.csv (形狀: (30000, 48))
已儲存 Y 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Y.csv (形狀: (30000, 48))
已儲存 Z 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Z.csv (形狀: (30000, 48))
已儲存 Voltage 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Voltage.csv (形狀: (30000, 48))
所有測量類型的資料已分開儲存完畢。


###**Face_Check

In [29]:
import pandas as pd
import os

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
measurement_types = ['X', 'Y', 'Z', 'Voltage']

print("正在載入並顯示每個分開儲存的 CSV 檔案的前5行和後5行：\n")

for m_type in measurement_types:
    output_file_name = f'Healthy_Merge_Clean_Table_{m_type}.csv'
    output_file_path = os.path.join(output_base_dir, output_file_name)

    if os.path.exists(output_file_path):
        print(f"--- {output_file_name} ---")
        df_loaded = pd.read_csv(output_file_path)
        print("前5行：")
        display(df_loaded.head())
        print("後5行：")
        display(df_loaded.tail())
        print("\n")
    else:
        print(f"錯誤：檔案 {output_file_path} 不存在。")

print("所有檔案檢查完畢。")

正在載入並顯示每個分開儲存的 CSV 檔案的前5行和後5行：

--- Healthy_Merge_Clean_Table_X.csv ---
前5行：


,202409302014_01_X,202409302014_02_X,202409302014_03_X,202409302014_04_X,202409302014_05_X,202409302014_06_X,202409302014_07_X,202409302014_08_X,202409302014_09_X,202409302014_10_X,...,202409302014_39_X,202409302014_40_X,202409302014_41_X,202409302014_42_X,202409302014_43_X,202409302014_44_X,202409302014_45_X,202409302014_46_X,202409302014_47_X,202409302014_48_X
0,0.053857,0.006437,-0.012931,-0.025556,-0.011177,-0.029158,0.024948,-0.018021,-0.018625,0.004698,...,-0.018339,-0.018820,-0.003166,0.019873,0.027487,-0.029371,-0.003316,0.017632,-0.011159,-0.004920
1,0.032555,0.007665,0.011685,0.016332,-0.015793,-0.051850,0.021355,-0.006662,-0.032508,0.015592,...,-0.003302,-0.007441,-0.001389,0.000500,0.005066,-0.009990,-0.029379,0.000773,-0.005031,-0.018377
2,0.004798,-0.000910,0.023469,-0.021005,-0.021959,-0.028541,-0.020189,0.013649,-0.025893,0.017264,...,0.015708,-0.007477,-0.000003,-0.011413,0.006435,0.012263,-0.014929,-0.011990,0.006876,-0.030491
3,-0.019530,-0.007906,-0.002403,-0.030216,-0.002180,0.002617,-0.017600,0.020219,-0.029153,0.020154,...,-0.009273,-0.001912,-0.042673,0.000390,-0.014527,0.027033,0.008128,-0.028621,0.003613,-0.006652
4,-0.004428,0.003494,-0.023142,-0.033537,0.027174,0.025222,-0.006737,0.023051,-0.017705,0.027609,...,-0.020363,-0.010665,-0.037857,-0.003853,-0.016170,0.007323,0.038054,-0.014440,-0.015312,-0.007826


後5行：


,202409302014_01_X,202409302014_02_X,202409302014_03_X,202409302014_04_X,202409302014_05_X,202409302014_06_X,202409302014_07_X,202409302014_08_X,202409302014_09_X,202409302014_10_X,...,202409302014_39_X,202409302014_40_X,202409302014_41_X,202409302014_42_X,202409302014_43_X,202409302014_44_X,202409302014_45_X,202409302014_46_X,202409302014_47_X,202409302014_48_X
29995,0.009914,0.014185,-0.029351,0.031321,0.011528,0.039559,0.009563,0.050527,0.024373,-0.005044,...,0.016484,-0.003026,0.006194,-0.029694,-0.012847,0.010255,-0.008182,0.012265,0.011893,-0.010491
29996,0.006148,0.009509,0.060023,0.026032,0.022297,0.024422,0.004305,0.038846,0.026338,-0.022072,...,0.029507,0.015866,-0.026538,-0.031725,0.005595,-0.007568,0.002320,0.033503,0.002971,-0.010445
29997,0.003009,0.027948,0.026799,-0.003476,0.014452,-0.014357,0.004005,0.025595,-0.018106,-0.007075,...,0.037716,0.028303,-0.017614,-0.010021,0.009148,-0.016398,0.017311,0.013075,0.000870,-0.031694
29998,-0.010613,0.015428,0.034973,-0.004707,-0.004110,-0.011784,0.012020,0.007027,-0.040779,-0.002027,...,0.015222,0.021730,-0.004587,0.003046,0.009030,0.000644,0.009845,0.001342,-0.007462,-0.020823
29999,-0.000942,-0.019569,-0.013964,-0.026634,-0.008798,-0.005263,-0.009914,-0.022176,-0.019426,-0.002655,...,-0.002495,0.005518,0.037897,0.037305,0.007367,0.007717,0.018241,-0.006729,0.010359,0.021872




--- Healthy_Merge_Clean_Table_Y.csv ---
前5行：


...



--- Healthy_Merge_Clean_Table_Voltage.csv ---
前5行：


,202409302014_01_Voltage,202409302014_02_Voltage,202409302014_03_Voltage,202409302014_04_Voltage,202409302014_05_Voltage,202409302014_06_Voltage,202409302014_07_Voltage,202409302014_08_Voltage,202409302014_09_Voltage,202409302014_10_Voltage,...,202409302014_39_Voltage,202409302014_40_Voltage,202409302014_41_Voltage,202409302014_42_Voltage,202409302014_43_Voltage,202409302014_44_Voltage,202409302014_45_Voltage,202409302014_46_Voltage,202409302014_47_Voltage,202409302014_48_Voltage
0,0.000443,0.000124,0.000761,0.002672,0.000443,0.001080,0.002354,0.000443,-0.057536,0.001080,...,-0.021220,0.000761,0.000761,0.003310,0.000761,-0.078242,0.001398,-0.000513,0.002672,0.025290
1,0.002672,0.001398,0.001080,0.002991,0.046953,0.001398,0.002991,0.000124,-0.037785,0.029113,...,0.003310,0.001398,0.001398,0.003628,0.000124,-0.100860,0.001398,0.000124,0.002672,-0.000513
2,0.002672,0.001080,0.001398,0.003628,0.092188,0.001398,0.002991,0.000124,-0.015167,0.082632,...,0.002672,0.001080,0.000761,0.002991,0.000443,-0.106276,0.002035,0.000761,0.002672,0.000761
3,0.002991,0.001080,0.001080,0.002991,0.114488,0.002672,0.003310,-0.000195,0.002991,0.110665,...,0.002991,0.002672,0.002035,0.002672,0.000443,-0.102453,0.000761,0.001080,0.002991,0.000443
4,0.002991,0.000761,0.001080,0.003310,0.125637,0.002035,0.002672,-0.000832,0.002672,0.120859,...,0.002672,0.071163,0.001398,0.003628,0.000124,-0.090029,0.001080,0.002354,0.003947,0.001398


後5行：


,202409302014_01_Voltage,202409302014_02_Voltage,202409302014_03_Voltage,202409302014_04_Voltage,202409302014_05_Voltage,202409302014_06_Voltage,202409302014_07_Voltage,202409302014_08_Voltage,202409302014_09_Voltage,202409302014_10_Voltage,...,202409302014_39_Voltage,202409302014_40_Voltage,202409302014_41_Voltage,202409302014_42_Voltage,202409302014_43_Voltage,202409302014_44_Voltage,202409302014_45_Voltage,202409302014_46_Voltage,202409302014_47_Voltage,202409302014_48_Voltage
29995,0.001717,0.001080,-0.049572,0.000443,0.001717,0.003310,0.001080,-0.093852,0.000443,0.001080,...,0.001717,0.000124,0.002991,0.000124,0.002672,0.002035,0.000124,0.002991,0.122770,0.002672
29996,0.002035,-0.000513,-0.023131,0.001717,0.001080,0.002672,0.000124,-0.101179,0.001398,0.001398,...,0.000124,0.001080,0.003628,0.000124,0.002991,0.001717,-0.000513,0.002672,0.110346,0.001717
29997,0.001398,0.000761,0.000761,0.000761,0.001717,0.002991,0.000124,-0.104046,0.001717,0.001080,...,0.000761,0.001080,0.003310,0.000761,0.002991,0.002035,0.000124,0.002991,0.092507,0.002672
29998,-0.000513,0.001398,0.003310,0.000443,0.001080,0.002991,0.000443,-0.094808,0.001398,0.001398,...,0.001080,0.001717,0.002991,0.000761,0.001717,0.001717,-0.000513,0.003310,0.074030,0.002991
29999,0.001080,0.001080,0.003310,0.001398,0.001080,0.003947,0.000443,-0.077287,0.000443,0.002035,...,0.002035,0.001398,0.003628,0.001398,-0.034281,0.001398,-0.000195,0.002672,0.053005,-0.014530




所有檔案檢查完畢。
